# 패키지, 데이터 로드

In [3]:
# 패키지 로드 
import pandas as pd 
from bs4 import BeautifulSoup  # html 파싱 
import re  # 정규표현식

In [4]:
# 데이터 로드 
file_path = '../../../data/mirae/2023BigFesta_NewsDataset_202304_5.xlsx'  
news = pd.read_excel(file_path)
news.head() # 확인 

,Unnamed: 0,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,19728,202304,20230401_002059,로이터(AI뉴스),사회,"트럼프의 뉴욕 변호사들, 긴고된 역사의 최신 사례",16.21,NaN,미국 |,<html>\n<head>\n<style>\n\n * {\n ...
1,19729,202304,20230401_002610,로이터(AI뉴스),경제,"캘리포니아, 2035년까지 판매되는 무거운 트럭 중 절반은 전기 트럭이어야 함 - ...",16.17,NaN,미국 |자동차 |캐나다 |소재 |트럭| 전기,<html>\n<head>\n<style>\n\n * {\n ...
2,19730,202304,20230401_010310,로이터(AI뉴스),기술,"애플, 영국의 모바일 브라우저 조사 결정에 대한 항소에서 승리",55.30,애플|,서비스 |영국 |미국 |소재 |유통 |EU |IT |모바일,<html>\n<head>\n<style>\n\n * {\n ...
3,19731,202304,20230401_002928,로이터(AI뉴스),사회,캐나다 로저스-쇼 합의 승인에 대한 반응,48.56,NaN,캐나다 |커뮤니케이션 |,<html>\n<head>\n<style>\n\n * {\n ...
4,19732,202304,20230401_005517,로이터(AI뉴스),경제,"미국, 2035년까지 중량물 트럭의 절반을 전기차로 요구하는 캘리포니아 계획 승인",16.40,NaN,자동차 |미국 |,<html>\n<head>\n<style>\n\n * {\n ...


In [5]:
news.drop('Unnamed: 0',axis = 1, inplace = True) # 불필요한 열 삭제 
news.head() # 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,202304,20230401_002059,로이터(AI뉴스),사회,"트럼프의 뉴욕 변호사들, 긴고된 역사의 최신 사례",16.21,NaN,미국 |,<html>\n<head>\n<style>\n\n * {\n ...
1,202304,20230401_002610,로이터(AI뉴스),경제,"캘리포니아, 2035년까지 판매되는 무거운 트럭 중 절반은 전기 트럭이어야 함 - ...",16.17,NaN,미국 |자동차 |캐나다 |소재 |트럭| 전기,<html>\n<head>\n<style>\n\n * {\n ...
2,202304,20230401_010310,로이터(AI뉴스),기술,"애플, 영국의 모바일 브라우저 조사 결정에 대한 항소에서 승리",55.30,애플|,서비스 |영국 |미국 |소재 |유통 |EU |IT |모바일,<html>\n<head>\n<style>\n\n * {\n ...
3,202304,20230401_002928,로이터(AI뉴스),사회,캐나다 로저스-쇼 합의 승인에 대한 반응,48.56,NaN,캐나다 |커뮤니케이션 |,<html>\n<head>\n<style>\n\n * {\n ...
4,202304,20230401_005517,로이터(AI뉴스),경제,"미국, 2035년까지 중량물 트럭의 절반을 전기차로 요구하는 캘리포니아 계획 승인",16.40,NaN,자동차 |미국 |,<html>\n<head>\n<style>\n\n * {\n ...


# 키워드 지정
- 아래의 키워드를 뉴스 본문 html 이 저장된 변수인 'CONTENT'변수에 2번 이상 포함하지 않을 경우 엔터테인먼트와 무관한 기사로 판단하여 제거한다.

In [8]:
keywords = [
    '방시혁','하이브','에스엠','이수만','와이지','양현석','제이와이피','박진영',
    
    '뉴진스','방탄소년단','르세라핌',
    '세븐틴','투모로우바이투게더',
    '프로미스나인','나나','이현','미드낫','엔하이픈',
    '백호','강동호','황민현','예하나','성연','지코',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    '블랙핑크','트레저','악뮤','악동뮤지션',
    '위너','지디','권지용',
    '젝스키스',
    
    '트와이스','엔믹스', '데이식스',
    'itzy','채령','류진','유나','예지'
    
    '스트레이키즈','투피엠','보이스토리',
    '야오천','니지유','엑스디너리 히어로즈',
    'boa',
    '에스파',
    '엑소',
    
    '샤이니','엔시티','엔시티 127','엔시티 드림',    
    '동방신기','슈퍼주니어','소녀시대','엑소'
    '레드벨벳','웨이션브이','슈퍼엠',
    '갓더비트','갓 더 비트','라이즈'

]
# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)
# 'PARSED CONTENT' 열에서 정규식 패턴을 검색하여 추출
filtered_df = news[news['CONTENT'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)]
# 특정 키워드가 2번 이상 포함된 행만 남기기
filtered_df = filtered_df[filtered_df['CONTENT'].apply(lambda text: sum(1 for keyword in keywords if keyword.lower() in text.lower()) >= 2)]


In [9]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 4 to 224164
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NEWS_YM      2000 non-null   int64  
 1   DATE_TIME    2000 non-null   object 
 2   WRITER       2000 non-null   object 
 3   SUBCATEGORY  2000 non-null   object 
 4   TITLE        2000 non-null   object 
 5   IMPORTANCE   2000 non-null   float64
 6   ITEM_NAME    1017 non-null   object 
 7   TAG_LIST     1632 non-null   object 
 8   CONTENT      2000 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 156.2+ KB


# 데이터 저장

In [10]:
# 데이터 저장
file_path = '../../../data/News/news_1filtered_202304.xlsx'  
filtered_df.to_excel(file_path, index=False)